In [32]:
%load_ext autoreload
%autoreload 2
#from src.VLLMServer import launch_command,get_client_dict,ask_llm,ask_llm_prompt
from src.code_utils import extract_code_block
from src.process_openmath_ja import parse_record,eval_answer
import yaml
from vllm import SamplingParams, LLM
import json

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
out_path="processed/data.jsonl"

In [7]:

#load dataset
from datasets import load_dataset
ds=load_dataset("kunishou/OpenMathInstruct-1-1.8m-ja",split="train",streaming=False)

#ds=ds.shuffle()

In [3]:
model_name = "microsoft/Phi-3-medium-128k-instruct"

llm = LLM(model=model_name, trust_remote_code=True,
          max_model_len=20000
          )

/home/setup/miniconda3/envs/llmeval/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


INFO 06-18 15:17:43 llm_engine.py:100] Initializing an LLM engine (v0.4.2) with config: model='microsoft/Phi-3-medium-128k-instruct', speculative_config=None, tokenizer='microsoft/Phi-3-medium-128k-instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=20000, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=microsoft/Phi-3-medium-128k-instruct)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 06-18 15:17:43 utils.py:660] Found nccl from library /home/setup/.config/vllm/nccl/cu12/libnccl.so.2.18.1
INFO 06-18 15:17:44 selector.py:27] Using FlashAttention-2 backend.
INFO 06-18 15:17:46 weight_utils.py:199] Using model weights format ['*.safetensors']
INFO 06-18 15:17:50 model_runner.py:175] Loading model weights took 26.1473 GB
INFO 06-18 15:17:53 gpu_executor.py:114] # GPU blocks: 13470, # CPU blocks: 1310
INFO 06-18 15:17:54 model_runner.py:937] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 06-18 15:17:54 model_runner.py:941] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 06-18 15:17:58 model_runner.py:1017] Graph capturing 

In [8]:
model_name="microsoft/Phi-3-medium-128k-instruct"
pre_command="""次の問題を解きなさい｡出力事項は以下のとおりである｡
#解説: 問題を解くための基本的な考え方を日本語で出力する｡
#Python: 計算プログラム｡以下の例を参考に､出力形式を守ること｡
```
#a+bの計算を行う
a=1
b=2
print(a+b)
```
#問題
"""

In [49]:
from tqdm import tqdm
batch_size=100
idx=0


for j in tqdm(range(int(len(ds)/batch_size))):

    prompts=[]
    answer_list=[]
    question_list=[]
    idx_list=[]

    for i in range(batch_size):
        record=ds[idx]
        idx+=1
        #問題文と回答の取得
        question,answer=parse_record(record)
        answer_list.append(answer)
        question_list.append(question)
        idx_list.append(idx)

        #promptの生成
        ja_text=pre_command+question
        prompt= f"""<|user|>
    {ja_text}<|end|>
    <|assistant|>"""
        prompts.append(prompt)


    #回答の生成
    outputs = llm.generate(
        prompts,
        sampling_params=SamplingParams(
            temperature=0.1,
            max_tokens=2048,
            repetition_penalty=1.2,
        )
    )

    #回答のチェック
    for i in range(len(outputs)):
        res=outputs[i].outputs[0].text.strip()
        res=res.replace("\'","`")

        #生成されたコードを実際にまわして評価する
        explanation_block=res[:res.find("```")]
        try:
            code_block=extract_code_block(res)
        except:
            continue


        answer=answer_list[i]
        question=question_list[i]
        #真偽判定
        is_correct_answer=eval_answer(code_block,answer,verbose=False)

        generated_prompt=explanation_block+"\n"+code_block
        if is_correct_answer:
            new_record={
                    "id":idx_list[i],
                    "question":question,
                    "prediction":generated_prompt,
                    "answer":answer,

            }
            with open(out_path,"a") as f:
                f.write(json.dumps(new_record,ensure_ascii=False)+"\n")

  0%|          | 0/18250 [00:00<?, ?it/s]
























































































































































































































































































































































  0%|          | 0/18250 [02:01<?, ?it/s]


IndexError: list index out of range